In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
pip install torch torchvision scikit-learn matplotlib grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39648 sha256=1e7277c08fa21087f1220caf86025b7f08dbf8ebdd9ba3aecf7aa554c96a3402
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [ ]:
https://drive.google.com/file/d/1krplHcJqXsQ7Yxk9gaAAs49Pc3DcnJML/view?usp=sharing

In [2]:
file_id = "1krplHcJqXsQ7Yxk9gaAAs49Pc3DcnJML"
!gdown "https://drive.google.com/uc?id={file_id}" -O caltech-101.zip

Downloading...
From (original): https://drive.google.com/uc?id=1krplHcJqXsQ7Yxk9gaAAs49Pc3DcnJML
From (redirected): https://drive.google.com/uc?id=1krplHcJqXsQ7Yxk9gaAAs49Pc3DcnJML&confirm=t&uuid=8199341a-6886-4bce-8842-7b9f4da132d8
To: /content/caltech-101.zip
100% 137M/137M [00:04<00:00, 29.3MB/s]


In [3]:
!unzip -q caltech-101.zip -d .


In [6]:
import os
import torch
from torch import nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

# Paths and Settings

In [8]:

data_dir = "/content/caltech-101"
batch_size = 32
num_classes = 101
num_epochs = 10
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset Loading and Transformations

In [19]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
num_classes = len(dataset.classes)  # Automatically detect the number of classes

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Model Selector

In [20]:

def select_model(model_name):
    if model_name == "resnet50":
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == "vgg19":
        model = models.vgg19(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    elif model_name == "efficientnet_b0":
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Unsupported model name")
    return model.to(device)


In [ ]:
# Select Pre-trained Model

In [21]:

model_name = "resnet50"  # Change to "vgg19" or "efficientnet_b0" as needed
model = select_model(model_name)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Loss and Optimizer

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop

In [ ]:

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/total:.4f}, Accuracy: {train_acc:.4f}")

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f"Validation Accuracy: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

Epoch 1/10, Loss: 3.4727, Accuracy: 0.2835
Validation Accuracy: 0.3720


# Load Best Model

In [ ]:

model.load_state_dict(torch.load("best_model.pth"))

# Grad-CAM Visualization

In [ ]:

def grad_cam_visualization(model, test_loader):
    target_layer = model.layer4[-1] if hasattr(model, 'layer4') else model.features[-1]  # Adjust based on architecture
    cam = GradCAM(model=model, target_layer=target_layer, use_cuda=(device.type == 'cuda'))

    for i, (inputs, labels) in enumerate(test_loader):
        if i >= 5:  # Limit to 5 visualizations
            break
        inputs = inputs.to(device)
        grayscale_cam = cam(input_tensor=inputs)
        for j in range(inputs.size(0)):
            img = inputs[j].cpu().permute(1, 2, 0).numpy()
            img = (img * 0.229 + 0.485).clip(0, 1)  # Reverse normalization
            cam_image = show_cam_on_image(img, grayscale_cam[j], use_rgb=True)
            plt.imshow(cam_image)
            plt.title(f"Label: {labels[j]} Prediction: {torch.argmax(model(inputs[j:j+1]), 1).item()}")
            plt.axis('off')
            plt.show()

grad_cam_visualization(model, test_loader)


# Grad-CAM Visualization

In [ ]:

def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    print("Classification Report:")
    print(classification_report(y_true, y_pred))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
evaluate_model(model, test_loader)

In [ ]:
# Grad-CAM
def grad_cam_test(model, image_path):
    target_layer = model.layer4[-1] if hasattr(model, 'layer4') else model.features[-1]
    cam = GradCAM(model=model, target_layer=target_layer, use_cuda=(device.type == 'cuda'))

    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)  # Apply the same transformations

    grayscale_cam = cam(input_tensor=input_tensor)[0]
    img = input_tensor[0].cpu().permute(1, 2, 0).numpy()
    img = (img * 0.229 + 0.485).clip(0, 1)  # Reverse normalization
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)

    plt.imshow(cam_image)
    plt.title("Grad-CAM Visualization")
    plt.axis('off')
    plt.show()

In [ ]:
# Example Usage
grad_cam_test(model, "path_to_image.jpg")  # Replace with the actual image path